<a href="https://colab.research.google.com/github/YOUR_USERNAME/ai-toolkit-rgba/blob/main/colab_webui_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎨 AI Toolkit Web UI on Colab

Run the full ai-toolkit training & generation web UI on **Colab's GPU** with public internet access.

Features:
- ✅ FLUX.2 Klein 4B/9B training
- ✅ RGBA transparent image support
- ✅ LoRA training & inference
- ✅ Public web UI access from anywhere
- ✅ Full Colab GPU acceleration

**Time to start**: ~5 minutes

## 📋 Setup

### Step 1️⃣: Check GPU (Run this first)

In [ ]:
import torch
import subprocess

# Check GPU
print("🖥️  GPU Info:")
print(f"   Device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU only'}")
print(f"   CUDA: {torch.version.cuda}")

if torch.cuda.is_available():
    print(f"   VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    print("\n✅ GPU available!")
else:
    print("\n⚠️  WARNING: No GPU detected! Training will be slow.")
    print("   Make sure to enable GPU in Runtime > Change runtime type > GPU")

### Step 2️⃣: Clone Repository

In [ ]:
import os
import subprocess

# Clone the repo
repo_url = "https://github.com/YOUR_USERNAME/ai-toolkit-rgba.git"  # ← UPDATE THIS
repo_path = "/content/ai-toolkit"

if not os.path.exists(repo_path):
    print(f"📥 Cloning from {repo_url}...")
    subprocess.run(["git", "clone", repo_url, repo_path], check=True)
    print("✅ Clone complete!")
else:
    print("✅ Repository already cloned")

os.chdir(repo_path)
print(f"📂 Working in: {os.getcwd()}")

### Step 3️⃣: Install Dependencies

⏱️ This takes ~3-5 minutes

In [ ]:
import subprocess
import sys

print("📦 Installing Python dependencies...")
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "pyngrok"], check=True)
print("   ✅ pyngrok")

# Install Node.js if not present
result = subprocess.run(["node", "--version"], capture_output=True)
if result.returncode != 0:
    print("   Installing Node.js...")
    subprocess.run(["apt-get", "update", "-qq"], capture_output=True)
    subprocess.run(["apt-get", "install", "-y", "-qq", "nodejs", "npm"], check=True)

print("   ✅ Node.js")

print("\n📦 Installing npm dependencies for UI...")
subprocess.run(["npm", "install", "--prefix", "ui"], capture_output=True, check=False)
print("   ✅ npm packages")

print("\n✅ All dependencies installed!")

### Step 4️⃣: Build Web UI

⏱️ This takes ~2-3 minutes

In [ ]:
import subprocess

print("🔨 Building Next.js UI...")
result = subprocess.run(
    ["npm", "run", "build", "--prefix", "ui"],
    capture_output=True,
    text=True
)

if result.returncode == 0:
    print("✅ Build successful!")
else:
    print("⚠️  Build warnings (this is usually fine):")
    if result.stderr:
        print(result.stderr[:500])

### Step 5️⃣: Start Web UI with Public Tunnel

🚀 This starts the server and creates a public URL

In [ ]:
import subprocess
import time
from pyngrok import ngrok
from threading import Thread
import os
import signal

UI_PORT = 8675
ui_process = None

def start_ui_server():
    """Start the Next.js UI server"""
    global ui_process
    os.chdir("/content/ai-toolkit")
    ui_process = subprocess.Popen(
        ["npm", "run", "start", "--prefix", "ui"],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )
    ui_process.wait()

print("🚀 Starting web UI server...")
ui_thread = Thread(target=start_ui_server, daemon=False)
ui_thread.start()

# Wait for server to start
print("⏳ Waiting for server to start (30 seconds)...")
for i in range(30):
    time.sleep(1)
    if i % 5 == 0:
        print(f"   {i}s...")

# Create ngrok tunnel
print("\n🌐 Creating public tunnel with ngrok...")
try:
    public_url = ngrok.connect(UI_PORT, "http")
    print("\n" + "="*70)
    print("✅ WEB UI IS LIVE!")
    print("="*70)
    print(f"\n🔗 PUBLIC URL:\n\n   {public_url}")
    print("\n" + "="*70)
    print("\n📝 You can now:")
    print("   • Open the URL in your browser")
    print("   • Create training jobs")
   print("   • Generate images with LoRA")
    print("   • Monitor GPU usage")
    print("\n⏱️  Keep this cell running to keep the server alive!")
    print("   (You can open new cells to run other commands)\n")
    
    # Keep server alive
    ui_thread.join()
    
except KeyboardInterrupt:
    print("\n⏹️  Server stopped")
    ngrok.kill()
    if ui_process:
        ui_process.terminate()
except Exception as e:
    print(f"❌ Error: {e}")
    ngrok.kill()

## 🎯 Using the Web UI

Once the server is running (Step 5):

1. **Copy the PUBLIC URL** from the output above
2. **Open it in your browser** (or from your phone!)
3. **Create a training job**:
   - Select FLUX.2 Klein 4B as the model
   - Upload training images (RGBA PNG for transparency)
   - Configure LoRA settings
   - Start training
4. **Monitor in real-time** - watch the GPU usage and sample generations

### Features Available

✅ **Training**
- FLUX.2 Klein 4B/9B models
- LoRA fine-tuning
- Custom RGBA VAE support
- Real-time sample generation

✅ **Generation**
- Apply trained LoRAs
- Control images support
- Transparent (RGBA) output
- Batch generation

✅ **Monitoring**
- GPU/VRAM usage
- Training loss curves
- Sample preview
- Job history

## 🛠️ Helper Commands

Run these in separate cells while the server is running:

In [ ]:
# Check current GPU usage
import subprocess

print("📊 Current GPU Status:")
result = subprocess.run(["nvidia-smi"], capture_output=True, text=True)
print(result.stdout)

In [ ]:
# Check training job status
import os
import json
from pathlib import Path

output_dir = Path("/content/ai-toolkit/output")
if output_dir.exists():
    print("📁 Training outputs:")
    for folder in sorted(output_dir.iterdir()):
        if folder.is_dir():
            print(f"\n   📂 {folder.name}")
            # Show latest sample
            samples = list(folder.glob("*.png"))
            if samples:
                latest = max(samples, key=lambda p: p.stat().st_mtime)
                print(f"      Latest sample: {latest.name}")
else:
    print("No training outputs yet")

## 💡 Tips

- **Keep the notebook running**: The server cell needs to stay active. You can open new cells to run commands.
- **URL is temporary**: The ngrok URL changes each session. It's unique to this Colab session.
- **VRAM**: Colab usually has ~16GB VRAM. Perfect for FLUX.2 Klein 4B!
- **Download results**: Check `/content/ai-toolkit/output/` for all generated files
- **GPU limits**: Colab free tier has usage limits. Paid Colab Pro has no limits.

## ❓ Troubleshooting

**"Can't connect to URL"**
- Make sure the server cell is still running
- Wait 30+ seconds for the server to fully start
- Try opening in an incognito tab

**"Module not found" errors**
- Re-run Step 3 (Install Dependencies)

**GPU out of memory**
- Restart runtime (Runtime > Restart runtime)
- Make sure you're using GPU T4 or better (not CPU)

**Server won't start**
- Check Step 4 build output for errors
- Restart the notebook (Runtime > Restart runtime)